Overview:
- Parsing using twitter api and alpha vantage
- Cleaning, EDA
- Models

Conclusion:


## Import and Functions


In [43]:
# Parsing
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from alpha_vantage.techindicators import TechIndicators

# Data processing
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS

# NLP
import re
import nltk
nltk.download('stopwords')
nltk.download('words')
words = set(nltk.corpus.words.words())
from nltk.probability import FreqDist


# Misc
import os
import glob


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gumo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Gumo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:

# Load key for tweepy
path = r'C:\Users\Gumo\Desktop\Git\Notebook\keys\tweepy.txt'
keys = []
with open(path, mode='r') as w:
    for line in w:
        keys.append(line.split(': ')[1].strip())
access_token = keys[3]
access_secret = keys[4]
consumer_key = keys[0]
consumer_secret = keys[1]
bearer_token = keys[2]

###### my id/name #####
myUserId = 1309643764172947456
myUsername = 'stockjanitor'

# Load Twitter API
# Twitter API v1 - auth keys (Cursor)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token,access_secret)

# create instance
twitterApi = tweepy.API(auth, wait_on_rate_limit=True)
# wait on rate limit, not to get errors on timing out

# verify credential
try:
    # see if credential works
    twitterApi.verify_credentials()
    print("twitterApi works")
except:
    print("Please fix me")



twitterApi works


In [8]:
# Variable - store Alpha Vantage API key
path =  r'C:\Users\Gumo\Desktop\Git\Notebook\keys\alphaVantage.txt'
with open(path, mode='r') as w:
    alphaKey = (w.readline())

In [48]:
# Function AlphaVantage
def alpha(key, symbol, function, period=None, typ = None, tim=None, outputsize = 'compact'):
    
    # Funciton - TimeSeries
    if function == 't':
        # Instance - TimeSeries Instance
        ts = TimeSeries(key = key, output_format='pandas')
        if period == 'd':
            data = ts.get_daily(symbol, outputsize=outputsize)[0]
        elif period == 'w':
            data = ts.get_weekly_adjusted(symbol)[0]
        elif period == 'm':
            data = ts.get_monthly_adjusted(symbol)[0]

    # Funciton - FundamentalData
    elif function == 'f':
        # Instance - FundamentalData Instance
        fd = FundamentalData(key, output_format='pandas')
        if period == 'q':
            if typ == 'i':
                data = fd.get_income_statement_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
        elif period =='a':
            if typ == 'i':
                data = fd.get_income_statement_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
    
    # Funciton - TechnicalIndicator            
    elif function =='ti':
        ti = TechIndicators(key,output_format='pandas')

        # reassign period into TI format
        if period == 'm':
            period = 'monthly'
        elif period =='w':
            period = 'weekly'
        elif period == 'd':
            period ='daily'

        if typ == 'rsi':
            data=ti.get_rsi(symbol,interval=period,time_period=tim,series_type='close')[0]
        elif typ =='so':
            data = ti.get_stoch(symbol,interval=period)[0]
            
    # Funciton - CC
    elif function == 'c':
        # Instance - Crypto
        cc = CryptoCurrencies(key, output_format='pandas')
        pass
    return data

In [10]:
###### Twitter FUNCTIONS ######
def tweetPost(text):
    try:
        twitterApi.update_status(text)
        print("bling bling~~")
    except:
        print("boom boom")
def tweetMedia(text, img):
    try:
        media = twitterApi.media_upload(img)
        twitterApi.update_status(text,media_ids=[media.media_id_string])
        print("bling bling~~")
    except:
        print("boom boom")
def tweetFriend(name):
    try:
        twitterApi.create_friendship(screen_name=name)
        print("bling bling~~")
    except:
        print("boom boom")
def tweetUnfriend(name):
    try:
        twitterApi.destroy_friendship(screen_name=name)
        print("bling bling~~")
    except:
        print("boom boom")

## Parsing

In [42]:
# set query1
query1 = "#twtr"
hour = '01'

date = '20220530'
time =hour+'00'
time2='2359'
fromdate =date+time
todate=date+time2

######################## Search 30 Day Queries ########################
# store search in tweets
queryResponse1 = twitterApi.search_30_day(label='30Days',query=query1,fromDate=fromdate,toDate=todate,maxResults=100)
# save tweets to list1
list1 = [[tweet.user.screen_name,tweet.text,tweet.created_at,tweet.user.location,tweet.source,tweet.favorite_count,tweet.quote_count,tweet.reply_count,tweet.retweet_count]for tweet in queryResponse1]
# convert tweet list into dataframe
df = pd.DataFrame(data=list1,columns=['user','text','time','location','source','like_count','quote_count','reply_count','retweet_count'])
# save to dataframe
df.to_csv('data/'+date+time+time2+query1+'.csv')
df.shape



######################## Search Recent queries ########################
# sincedate=''
# queryResponse1 = twitterApi.search_tweets(q=query1,until=sincedate,count=100)
# # items = 5, retrieve 5 tweets
# list1 = [[tweet.user.screen_name,tweet.text,tweet.created_at,tweet.user.location,tweet.source,tweet.favorite_count,tweet.retweet_count]for tweet in queryResponse1]
# # convert tweet list into dataframe
# df = pd.DataFrame(data=list1,columns=['user','text','time','location','source','like_count','retweet_count'])
# # save to dataframe
# df.to_csv('data/'+sincedate+query1+'.csv')
# df.shape

(54, 9)

In [52]:
# Variable Size (compact or full)
outputsize = 'compact'

# Variable Stock
symbol = 'twtr'

# Variable Function - t, f, c, ti (timeseries, financialdata, crypto, technical indicator)
function = 't'

# Variable Period
# TimeSeries: d,w,m,i (daily, weekly, monthly, interval(mins))
# Fundamental: q,a (quarterly,annually)
# Technical: d,w,m, (interval(mins))
period = 'd'

# Varible - Type
# Fundamental: i,b,c (income statement, balance sheet, cashflow)
# Technical: so,rsi
typ ='so'

# Varible - Time
tim = 60

info = alpha(alphaKey,symbol,function,period=period,typ = typ,tim=tim, outputsize='full')
info.head()



,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2022-05-31,40.07,40.3500,39.42,39.60,11154668.0
2022-05-27,39.57,40.7700,39.48,40.17,16789855.0
2022-05-26,38.52,39.5899,38.23,39.52,22968398.0
2022-05-25,36.06,37.3700,36.02,37.16,23262766.0
2022-05-24,36.94,37.2026,35.40,35.76,22662972.0


In [55]:
# save to local stock data
info.to_csv('data/'+symbol.upper()+'stockPrice.csv')

In [56]:
info.tail()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2013-11-13,41.03,42.87,40.760,42.60,8688300.0
2013-11-12,43.66,43.78,41.830,41.90,6316700.0
2013-11-11,40.50,43.00,39.400,42.90,16113900.0
2013-11-08,45.93,46.94,40.685,41.65,27925300.0
2013-11-07,45.10,50.09,44.000,44.90,117701600.0


## Outline